In [1]:
import pickle
import csv
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
import os
import datetime as dt
import pandas as pd
from scipy import stats
from pyearth import Earth#conda install sklearn-contrib-py-earth
from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor

np.warnings.filterwarnings('ignore') #Runtime warning

In [2]:
#Header of csv: 'Date[0],Time[1],Depth of Airgun(m)[2],Depth of Reciever(m)[3],X Airgun[4],Y Airgun[5],Z Airgun[6],X_R1[7],Y_R1[8],Z_R1[9],SEL1[10],RMS1[11],SEL2[12],RMS2[13],SEL3[14],RMS3[15],SEL4[16],RMS4[17],SEL_full[18],RMS_full[19],T90_1[20],T90_2[21],T90_3[22],T90_4[23],T90_full[24]
def prepData(x_data,y_data,path):
    csv_train_path = path
    files = os.listdir(csv_train_path)
    dataTrain = {}
    x_data2 = x_data
    y_data2 = y_data
    for filename in sorted(files):
        dirname = csv_train_path + filename
        dataTrain = pd.read_csv(dirname)
        dataTrain = dataTrain.rename(columns={'Depth of Airgun(m)': 'ag_depth','Depth of Reciever(m)': 'hyd_depth'})
        dataTrain = dataTrain[(dataTrain.hyd_depth != -99999)]    
        ag_x = dataTrain['X Airgun']
        ag_y = dataTrain['Y Airgun']
        ag_z = dataTrain['Z Airgun']
        hyd_x = dataTrain['X_R1'] #relative to ag_x
        hyd_y = dataTrain['Y_R1'] #relative to ag_y
        hyd_z = dataTrain['Z_R1'] #relative to ag_z
        r_train = np.sqrt(np.square(hyd_x) + np.square(hyd_y) + np.square(hyd_z))
        logr_train = np.log10(np.sqrt(np.square(hyd_x) + np.square(hyd_y) + np.square(hyd_z)))
        dataTrain = dataTrain.assign(r_train = r_train,logr_train = logr_train,filename = filename)
        dataTrain = dataTrain.dropna(axis=0, how='any')
        
        #outliers
        sel_dif = {}
        sel_dif = dataTrain['SEL_full'].diff()
        sel_dif2 = sel_dif.dropna(axis=0, how='any')
        sel_thres = np.abs(np.mean(np.abs(sel_dif2)))
        sel_dif = sel_dif[np.abs(sel_dif.values) < sel_thres]
        dataTrain = dataTrain.loc[sel_dif.index]
        
 #       rms_dif = {}
 #       rms_dif = dataTrain['RMS_full'].diff()
 #       rms_dif2 = rms_dif.dropna(axis=0, how='any')
 #       rms_thres = np.abs(np.mean(np.abs(rms_dif2)))
 #       rms_dif = rms_dif[np.abs(rms_dif.values) < rms_thres]
 #       dataTrain = dataTrain.loc[rms_dif.index]        

        x_data2 = x_data2.append(dataTrain[['ag_depth', 'hyd_depth','r_train','logr_train']],ignore_index=True)
        y_data2 = y_data2.append(dataTrain[['SEL_full','RMS_full']], ignore_index=True)
        #data = data.append(dataTrain[['filename','ag_depth', 'hyd_depth','logr_train','r_train','RMS_full','SEL_full']],ignore_index=True)
    return [x_data2,y_data2]

def pickle2(filename,x_data,y_data):
    outfile = open(filename,'wb')
    pickle.dump({'X': x_data, 'SEL': y_data['SEL_full'], 'RMS': y_data['RMS_full']},outfile)
    outfile.close()
    return;

In [4]:
#remove outliers and put data in frames
#path5 = r'D:\Machine Learning\NewCSV\CSV1\Line_05_(RMS_and_SEL)\\'
#path6 = r'D:\Machine Learning\NewCSV\CSV1\Line_06_(RMS_and_SEL)\\'
#path7 = r'D:\Machine Learning\NewCSV\CSV1\Line_07_(RMS_and_SEL)\\'
#[x_data,y_data] = prepData(pd.DataFrame(),pd.Series(),path5)
#pickle2('line05.pickle',x_data,y_data)
#[x_data,y_data] = prepData(x_data,y_data,path6)
#[x_data,y_data] = prepData(x_data,y_data,path7)
#pickle2('3lines.pickle',x_data,y_data)

In [4]:
#SEL
#draw from the pickle
infile = open('line05.pickle','rb')
#infile = open('3lines.pickle','rb')
data = pickle.load(infile, encoding='bytes')
x_train, x_test, y_train, y_test = train_test_split(data['X'],data['SEL'],test_size=0.1,shuffle=True)

In [5]:
print("Models for SEL")
print("=====================Linear=====================")
reg = linear_model.LinearRegression(fit_intercept=True, normalize=False)
# Train the model using the training sets
linreg_model = reg.fit(x_train, y_train)
print('Coefficients (slope): \n', reg.coef_)
print('Intercept: \n', reg.intercept_)

 
y_pred = linreg_model.predict(x_test)

R2 = reg.score(x_test, y_test)
print('R^2 score: %.2f \n' %R2)
err_total = mean_squared_error(y_test, y_pred)
print("Total Mean squared error: %.2f \n" %err_total)
 
x_test_subset = x_test[100:200]
y_test_subset = y_test[100:200]
y_pred_subset = linreg_model.predict(x_test_subset)
err_subspace = mean_squared_error(y_test_subset, y_pred_subset)
print("Subspace Mean squared error: %.2f \n" %err_subspace)

print("================2nd degree poly================")#2nd degree polynomial
poly = PolynomialFeatures(degree=2)
reg2 = linear_model.LinearRegression(fit_intercept=True, normalize=False)
x2_train = poly.fit_transform(x_train)
x2_test = poly.fit_transform(x_test)

linreg_model2 = reg2.fit(x2_train, y_train)
print('Coefficients (slope): \n', reg2.coef_)
print('Intercept: \n', reg2.intercept_)
 
y_pred = linreg_model2.predict(x2_test)

R2 = reg2.score(x2_test, y_test)
print('R^2 score: %.2f \n' %R2)
err_total = mean_squared_error(y_test, y_pred)
print("Total Mean squared error: %.2f \n" %err_total)
 
x2_test_subset = x2_test[100:200]
y_test_subset = y_test[100:200]
y_pred_subset = linreg_model2.predict(x2_test_subset)
err_subspace = mean_squared_error(y_test_subset, y_pred_subset)
print("Subspace Mean squared error: %.2f \n" %err_subspace)

print("================3rd degree poly================")#3rd degree polynomial
poly = PolynomialFeatures(degree=3)
reg2 = linear_model.LinearRegression(fit_intercept=True, normalize=False)
x2_train = poly.fit_transform(x_train)
x2_test = poly.fit_transform(x_test)

linreg_model2 = reg2.fit(x2_train, y_train)
print('Coefficients (slope): \n', reg2.coef_)
print('Intercept: \n', reg2.intercept_)
 
y_pred = linreg_model2.predict(x2_test)

R2 = reg2.score(x2_test, y_test)
print('R^2 score: %.2f \n' %R2)
err_total = mean_squared_error(y_test, y_pred)
print("Total Mean squared error: %.2f \n" %err_total)
 
x2_test_subset = x2_test[100:200]
y_test_subset = y_test[100:200]
y_pred_subset = linreg_model2.predict(x2_test_subset)
err_subspace = mean_squared_error(y_test_subset, y_pred_subset)
print("Subspace Mean squared error: %.2f \n" %err_subspace)

Models for SEL
=====================Linear=====================
Coefficients (slope): 
 [-4.96100217e-03  9.07722445e-04  5.95788505e-04 -9.65720800e+00]
Intercept: 
 200.0665743409653
R^2 score: 0.62 

Total Mean squared error: 8.65 

Subspace Mean squared error: 5.89 

================2nd degree poly================
Coefficients (slope): 
 [ 0.00000000e+00 -1.50089703e-02 -4.47643821e-03 -7.64537251e-01
 -7.19318215e+02 -1.05320375e-05  1.98311520e-05 -5.20283150e-07
  3.06679181e-03 -8.13255622e-06 -1.31768844e-09  8.06725557e-04
 -4.35403702e-06  1.84071580e-01  1.72332080e+02]
Intercept: 
 999.5956956796979
R^2 score: 0.66 

Total Mean squared error: 7.80 

Subspace Mean squared error: 5.34 

================3rd degree poly================
Coefficients (slope): 
 [ 5.28552320e-03 -1.57992528e+00  1.46342003e+00 -1.49079850e+02
 -6.26543432e+02 -1.14735713e-04  3.50465803e-04  1.41585063e-03
  1.27090957e+00 -2.25371241e-04 -1.27489621e-03 -1.17979949e+00
  3.13802782e-03  7.344485

In [ ]:
#this takes a while
#mars = Earth()
#mars.fit(x_train,y_train)
#
#filename = 'mars_sel.pickle'
#outfile = open(filename,'wb')
#
#pickle.dump(mars,outfile)
#outfile.close()

In [6]:
print("=======Multivariate Adaptive Regression Splines=======")
#https://contrib.scikit-learn.org/py-earth/content.html
infile = open('mars_sel.pickle','rb')
mars = pickle.load(infile)
y_pred = mars.predict(x_test)

R2 = mars.grsq_#MARS score equivalent
print('R^2 score: %.2f \n' %R2)
err_total = mean_squared_error(y_test, y_pred)
print("Total Mean squared error: %.2f \n" %err_total)
 
x_test_subset = x_test[100:200]
y_test_subset = y_test[100:200]
y_pred_subset = mars.predict(x_test_subset)
err_subspace = mean_squared_error(y_test_subset, y_pred_subset)
print("Subspace Mean squared error: %.2f \n" %err_subspace)

=======Multivariate Adaptive Regression Splines=======
R^2 score: 0.66 

Total Mean squared error: 7.85 

Subspace Mean squared error: 5.47 



In [7]:
#RMS
#draw from the pickle
infile = open('line05.pickle','rb')
#infile = open('3lines.pickle','rb')
data = pickle.load(infile, encoding='bytes')
x_train, x_test, y_train, y_test = train_test_split(data['X'],data['RMS'],test_size=0.1,shuffle=True)

In [8]:
print("Models for RMS")
print("=====================Linear=====================")
reg = linear_model.LinearRegression(fit_intercept=True, normalize=False)
# Train the model using the training sets
linreg_model = reg.fit(x_train, y_train)
print('Coefficients (slope): \n', reg.coef_)
print('Intercept: \n', reg.intercept_)

 
y_pred = linreg_model.predict(x_test)

R2 = reg.score(x_test, y_test)
print('R^2 score: %.2f \n' %R2)
err_total = mean_squared_error(y_test, y_pred)
print("Total Mean squared error: %.2f \n" %err_total)
 
x_test_subset = x_test[100:200]
y_test_subset = y_test[100:200]
y_pred_subset = linreg_model.predict(x_test_subset)
err_subspace = mean_squared_error(y_test_subset, y_pred_subset)
print("Subspace Mean squared error: %.2f \n" %err_subspace)

print("================2nd degree poly================")#2nd degree polynomial
poly = PolynomialFeatures(degree=2)
reg2 = linear_model.LinearRegression(fit_intercept=True, normalize=False)
x2_train = poly.fit_transform(x_train)
x2_test = poly.fit_transform(x_test)

linreg_model2 = reg2.fit(x2_train, y_train)
print('Coefficients (slope): \n', reg2.coef_)
print('Intercept: \n', reg2.intercept_)
 
y_pred = linreg_model2.predict(x2_test)

R2 = reg2.score(x2_test, y_test)
print('R^2 score: %.2f \n' %R2)
err_total = mean_squared_error(y_test, y_pred)
print("Total Mean squared error: %.2f \n" %err_total)
 
x2_test_subset = x2_test[100:200]
y_test_subset = y_test[100:200]
y_pred_subset = linreg_model2.predict(x2_test_subset)
err_subspace = mean_squared_error(y_test_subset, y_pred_subset)
print("Subspace Mean squared error: %.2f \n" %err_subspace)

print("================3rd degree poly================")#3rd degree polynomial
poly = PolynomialFeatures(degree=3)
reg2 = linear_model.LinearRegression(fit_intercept=True, normalize=False)
x2_train = poly.fit_transform(x_train)
x2_test = poly.fit_transform(x_test)

linreg_model2 = reg2.fit(x2_train, y_train)
print('Coefficients (slope): \n', reg2.coef_)
print('Intercept: \n', reg2.intercept_)
 
y_pred = linreg_model2.predict(x2_test)

R2 = reg2.score(x2_test, y_test)
print('R^2 score: %.2f \n' %R2)
err_total = mean_squared_error(y_test, y_pred)
print("Total Mean squared error: %.2f \n" %err_total)
 
x2_test_subset = x2_test[100:200]
y_test_subset = y_test[100:200]
y_pred_subset = linreg_model2.predict(x2_test_subset)
err_subspace = mean_squared_error(y_test_subset, y_pred_subset)
print("Subspace Mean squared error: %.2f \n" %err_subspace)

Models for RMS
=====================Linear=====================
Coefficients (slope): 
 [-6.77050283e-03  1.22143743e-03  1.82391784e-03 -1.37410540e+01]
Intercept: 
 215.0700278259755
R^2 score: 0.45 

Total Mean squared error: 20.60 

Subspace Mean squared error: 20.67 

================2nd degree poly================
Coefficients (slope): 
 [ 0.00000000e+00  8.62743871e-02 -9.18723674e-02 -1.33843730e+00
 -1.35875713e+03 -1.79620360e-05  3.00207262e-05  3.10674470e-06
 -2.81680954e-02 -8.85725077e-06 -2.87154396e-06  2.40536392e-02
 -7.53751253e-06  3.20799203e-01  3.23784074e+02]
Intercept: 
 1732.4160939998087
R^2 score: 0.50 

Total Mean squared error: 18.68 

Subspace Mean squared error: 17.85 

================3rd degree poly================
Coefficients (slope): 
 [ 2.18385329e-06 -1.88996882e+00  1.52886906e+00  2.77572022e+01
 -4.01287267e+02  2.55458090e-04 -2.41239735e-04  2.61355205e-03
  1.62159747e+00  3.28158577e-05 -1.90498861e-03 -1.30613204e+00
 -3.67136785e-03 -1.9

In [ ]:
#this takes a while

#mars = Earth()
#mars.fit(x_train,y_train)
#
#filename = 'mars_rms.pickle'
#outfile = open(filename,'wb')
#
#pickle.dump(mars,outfile)
#outfile.close()
#

In [9]:
print("=======Multivariate Adaptive Regression Splines=======")
#https://contrib.scikit-learn.org/py-earth/content.html
infile = open('mars_rms.pickle','rb')
mars = pickle.load(infile)
y_pred = mars.predict(x_test)

R2 = mars.grsq_#MARS score equivalent
print('R^2 score: %.2f \n' %R2)
err_total = mean_squared_error(y_test, y_pred)
print("Total Mean squared error: %.2f \n" %err_total)
 
x_test_subset = x_test[100:200]
y_test_subset = y_test[100:200]
y_pred_subset = mars.predict(x_test_subset)
err_subspace = mean_squared_error(y_test_subset, y_pred_subset)
print("Subspace Mean squared error: %.2f \n" %err_subspace)

=======Multivariate Adaptive Regression Splines=======
R^2 score: 0.47 

Total Mean squared error: 19.42 

Subspace Mean squared error: 18.03 



In [3]:
#whole data set
#path1 = r'D:\Machine Learning\NewCSV\CSV1\Line_01_(RMS_and_SEL)\\'
#path2 = r'D:\Machine Learning\NewCSV\CSV1\Line_02_(RMS_and_SEL)\\'
#path3 = r'D:\Machine Learning\NewCSV\CSV1\Line_03_(RMS_and_SEL)\\'
#path4 = r'D:\Machine Learning\NewCSV\CSV1\Line_04_(RMS_and_SEL)\\'
#path5 = r'D:\Machine Learning\NewCSV\CSV1\Line_05_(RMS_and_SEL)\\'
#path6 = r'D:\Machine Learning\NewCSV\CSV1\Line_06_(RMS_and_SEL)\\'
#path7 = r'D:\Machine Learning\NewCSV\CSV1\Line_07_(RMS_and_SEL)\\'
#path8 = r'D:\Machine Learning\NewCSV\CSV1\Line_08_(RMS_and_SEL)\\'
#path9 = r'D:\Machine Learning\NewCSV\CSV1\Line_09_(RMS_and_SEL)\\'
#path10 = r'D:\Machine Learning\NewCSV\CSV1\Line_10_(RMS_and_SEL)\\'
#path11 = r'D:\Machine Learning\NewCSV\CSV1\Line_11_(RMS_and_SEL)\\'
#pathAT = r'D:\Machine Learning\NewCSV\CSV1\Line_AT_(RMS_and_SEL)\\'
#
#[x_data,y_data] = prepData(pd.DataFrame(),pd.Series(),path1)
#[x_data,y_data] = prepData(x_data,y_data,path2)
#[x_data,y_data] = prepData(x_data,y_data,path3)
#[x_data,y_data] = prepData(x_data,y_data,path4)
#[x_data,y_data] = prepData(x_data,y_data,path5)
#[x_data,y_data] = prepData(x_data,y_data,path6)
#[x_data,y_data] = prepData(x_data,y_data,path7)
#[x_data,y_data] = prepData(x_data,y_data,path8)
#[x_data,y_data] = prepData(x_data,y_data,path9)
#[x_data,y_data] = prepData(x_data,y_data,path10)
#[x_data,y_data] = prepData(x_data,y_data,path11)
#[x_data,y_data] = prepData(x_data,y_data,pathAT)
#pickle2('all_lines.pickle',x_data,y_data)